In [1]:
import json

from model_handler.utils import ast_parse

/root/venvs/gorilla/lib/python3.10/site-packages/tree_sitter/__init__.py:36: FutureWarning: Language.build_library is deprecated. Use the new bindings instead.
  warn("{} is deprecated. Use {} instead.".format(old, new), FutureWarning)
/root/venvs/gorilla/lib/python3.10/site-packages/tree_sitter/__init__.py:36: FutureWarning: Language(path, name) is deprecated. Use Language(ptr, name) instead.
  warn("{} is deprecated. Use {} instead.".format(old, new), FutureWarning)


In [54]:
path = "/root/gorilla/berkeley-function-call-leaderboard/result/databricks-dbrx-instruct-tmp0/gorilla_openfunctions_v1_test_parallel_multiple_function.json"
generations = []

with open(path, "r") as f:
    for line in f:
        generations.append(json.loads(line))

In [61]:
def remove_duplicates(json_list):
    unique_json_list = []
    unique_json_strings = set()

    for json_obj in json_list:
        json_string = json.dumps(json_obj, sort_keys=True)
        if json_string not in unique_json_strings:
            unique_json_list.append(json_obj)
            unique_json_strings.add(json_string)

    return unique_json_list

In [56]:
def bfcl_format(tool_calls):
    tool_strs = []
    for tool_call in tool_calls:
        tool_name = list(tool_call.keys())[0]
        tool_args = list(tool_call.values())[0]
        args_string = ', '.join([f"{key}='{value}'" if isinstance(value, str) else f"{key}={value}" for key, value in tool_args.items()])
        tool_str = f'{tool_name}({args_string})'
        tool_strs.append(tool_str)
    result = '[' + ', '.join(tool_strs) + ']'
    return result

In [57]:
generations[2]["result"]

'[circle.calculate_area(radius=5), circle.calculate_circumference(diameter=10), circle.calculate_circumference(diameter=10)]'

In [62]:
new_generations = []
for i, gen in enumerate(generations):
    result = gen["result"]
    if "error" in result:
        new_generations.append(gen)
        continue
    tool_calls = ast_parse(result, "Python")
    tool_calls = remove_duplicates(tool_calls)
    new_result = bfcl_format(tool_calls)
    gen["result"] = new_result
    new_generations.append(gen)

In [65]:
write_path = "/root/gorilla/berkeley-function-call-leaderboard/result/databricks-dbrx-instruct-tmp0/gorilla_openfunctions_v1_test_parallel_multiple_function_dedup.json"

with open(write_path, "w") as f:
    for new_gen in generations:
        f.write(json.dumps(new_gen) + "\n")

In [1]:
from model_handler.outlines_vllm_handler_v2 import OutlinesVllmHandler


In [2]:
model_name = "meta-llama/Meta-Llama-3-70B-Instruct"
temperature = 0
top_p = 1
max_tokens = 150
handler = OutlinesVllmHandler(model_name, temperature, top_p, max_tokens)

In [4]:
if "gen_mode" in vars(handler):
    handler.gen_mode = gen_mode

{'model_style': <ModelStyle.Outlines: 'outlines'>,
 'n_tool_calls': 1,
 'gen_mode': 'conditional',
 'model_name': 'meta-llama/Meta-Llama-3-70B-Instruct',
 'temperature': 0,
 'top_p': 1,
 'max_tokens': 150,
 'base_url': 'http://localhost:8000/v1',
 'api_key': '-',
 'tool': <tool_use.tools.Tool at 0x7fd6ec12e8c0>,
 'idx': 0,
 'solutions': None}

In [17]:
import json
path = "/root/gorilla/berkeley-function-call-leaderboard/outputs/databricks_dbrx-instruct__conditional_0.7_1200/fingerprint.jsonl"

outputs = {}
with open(path, "r") as f:
    for line in f:
        outputs |= json.loads(line)

In [18]:
outputs

{'model': 'databricks/dbrx-instruct',
 'temperature': 0.7,
 'top_p': 1,
 'max_tokens': 1200,
 'gen_mode': 'conditional',
 'limit': 2,
 'limit_start': 0}